In [ ]:
from geopy.geocoders import Nominatim
import folium
import time

# O buscador de endereços (agent name pode ser qualquer um)
geolocator = Nominatim(user_agent="yakult_logistica_v2")

print("✅ Cérebro geográfico ativado!")

In [ ]:
# Célula 2 atualizada para América do Sul
pedidos = [
    {"local": "Yakult, Lorena, SP, Brazil", "garrafas": 0, "tipo": "fabrica"},
    {"local": "Curitiba, PR, Brazil", "garrafas": 500000, "tipo": "entrega"},
    {"local": "Foz do Iguaçu, PR, Brazil", "garrafas": 300000, "tipo": "entrega"},
    {"local": "Buenos Aires, Argentina", "garrafas": 800000, "tipo": "entrega"},
    {"local": "Mendoza, Argentina", "garrafas": 200000, "tipo": "entrega"},
    {"local": "Santiago, Chile", "garrafas": 200000, "tipo": "entrega"}
]

pontos_processados = []

print("🌍 Iniciando mapeamento internacional...")
for pedido in pedidos:
    loc = geolocator.geocode(pedido['local'])
    if loc:
        pontos_processados.append({
            "coords": [loc.latitude, loc.longitude],
            "nome": pedido['local'],
            "carga": pedido['garrafas'],
            "tipo": pedido['tipo']
        })
        print(f"OK: {pedido['local']}")

In [ ]:
# Criar o mapa
mapa = folium.Map(location=[-23.18, -45.88], zoom_start=8, tiles='OpenStreetMap')

coords_linha = []

for p in pontos_processados:
    coords_linha.append(p['coords'])
    
    # Personalização: Fábrica é Vermelha, Entrega é Azul
    cor = 'red' if p['tipo'] == 'fabrica' else 'blue'
    icone = 'industry' if p['tipo'] == 'fabrica' else 'truck'
    
    # Criando o balão de informações (Popup)
    texto_popup = f"<b>Destino:</b> {p['nome']}<br><b>Carga:</b> {p['carga']:,} garrafas"
    
    folium.Marker(
        location=p['coords'],
        popup=folium.Popup(texto_popup, max_width=300),
        icon=folium.Icon(color=cor, icon=icone, prefix='fa')
    ).add_to(mapa)

# Desenha a linha da rota
folium.PolyLine(coords_linha, color="darkblue", weight=3, opacity=0.8).add_to(mapa)

mapa.fit_bounds(coords_linha)
mapa

In [ ]:
# Célula 4: Relatório Financeiro Corrigido
km_por_litro = 3.5
preco_diesel = 6.00

def calcular_custos(pontos):
    distancia_total = 0
    
    for i in range(len(pontos) - 1):
        p1 = pontos[i]['coords']
        p2 = pontos[i+1]['coords']
        
        # A fórmula corrigida com ** para potência
        lat_diff = (p1[0] - p2[0])**2
        lon_diff = (p1[1] - p2[1])**2
        d = (lat_diff + lon_diff)**0.5 * 111
        
        distancia_total += d

    litros_gastos = distancia_total / km_por_litro
    custo_total = litros_gastos * preco_diesel
    
    print(f"📊 RELATÓRIO LOGÍSTICO YAKULT")
    print(f"-------------------------------")
    print(f"🛣️ Distância Total Estimada: {distancia_total:.2f} km")
    print(f"⛽ Diesel Necessário: {litros_gastos:.2f} L")
    print(f"💰 Custo Estimado de Combustível: R$ {custo_total:.2f}")
    print(f"-------------------------------")

# Executar a função
calcular_custos(pontos_processados)


In [ ]:
# Sua chave da TomTom que funcionou
SUA_CHAVE_TOMTOM = "wr1VOEhYoqbYwSazBcqJJ34zRwN2kb9R"

# Forma atualizada de adicionar o tráfego
folium.TileLayer(
    tiles=f"https://api.tomtom.com/traffic/map/4/tile/flow/relative0/{{z}}/{{x}}/{{y}}.png?key={SUA_CHAVE_TOMTOM}",
    attr="TomTom Traffic",
    name="Trânsito em Tempo Real",
    overlay=True,
    control=True,
    opacity=0.7
).add_to(mapa)

# Adiciona o botão para você ligar e desligar o trânsito no canto do mapa
folium.LayerControl().add_to(mapa)

mapa

In [ ]:
from routingpy import OSRM
import folium

# Usaremos o servidor público do OSRM (Open Source Routing Machine)
router = OSRM()

def desenhar_rota_real(mapa_obj, pontos):
    print("Calculando rotas pelas rodovias... aguarde.")
    
    # Pegamos os pontos que você já localizou (Lorena, SJC, Campinas...)
    coords_osrm = [[p['coords'][1], p['coords'][0]] for p in pontos]
    
    # Solicita a rota real (caminho das estradas)
    route = router.directions(locations=coords_osrm, profile='car')
    
    # 1. Tempo Estimado (ETA)
    horas = route.duration / 3600
    minutos = (horas - int(horas)) * 60
    distancia_km = route.distance / 1000
    
    print(f"⏱️ Tempo Total Estimado: {int(horas)}h {int(minutos)}min")
    print(f"🛣️ Distância Real por Rodovias: {distancia_km:.2f} km")

    # 2. Desenha a rota real no mapa (ela vai fazer as curvas da Dutra!)
    # Invertemos de volta para [lat, lon] que o Folium usa
    caminho_estrada = [[c[1], c[0]] for c in route.geometry]
    
    folium.PolyLine(caminho_estrada, color="green", weight=5, opacity=0.8, 
                    tooltip="Rota Otimizada Pelas Rodovias").add_to(mapa_obj)

# Chame a função para atualizar seu mapa
desenhar_rota_real(mapa, pontos_processados)
mapa

In [ ]:
# Célula 7: Inteligência de Desvio e Expansão Continental

def simulador_inteligente_yakult(origem_nome, destino_nome):
    # Simulamos o tempo baseado no trânsito que você viu na TomTom
    tempo_dutra = 125 # minutos (com trânsito)
    tempo_carvalho_pinto = 95 # minutos (mais livre)
    
    print(f"🚛 ANÁLISE DE ROTA: {origem_nome} -> {destino_nome}")
    print(f"--------------------------------------------------")
    
    if tempo_dutra > tempo_carvalho_pinto:
        print(f"⚠️ Alerta: Dutra com lentidão detectada!")
        print(f"✅ DECISÃO: Sugerindo desvio pela Carvalho Pinto.")
        print(f"⏱️ Economia estimada: {tempo_dutra - tempo_carvalho_pinto} minutos.")
    else:
        print(f"✅ DECISÃO: Rota principal (Dutra) segue sendo a mais rápida.")

# Executa o simulador
simulador_inteligente_yakult("São José dos Campos", "Lorena")




In [ ]:
def calcular_custo_internacional(pontos):
    print(f"💰 ANÁLISE DE CUSTO TRANSFRONTEIRIÇA")
    print(f"--------------------------------------")
    
    distancia_total = 0
    custo_total_reais = 0
    
    # Preços aproximados (convertidos para Real)
    PRECO_DIESEL_BR = 6.00
    PRECO_DIESEL_AR = 4.80  # Diesel costuma ser mais barato na Argentina
    PRECO_DIESEL_CH = 6.50  # No Chile é um pouco mais caro
    
    KM_LITRO = 3.5

    for i in range(len(pontos) - 1):
        p1 = pontos[i]
        p2 = pontos[i+1]
        
        # Cálculo de distância simples entre os pontos
        lat_diff = (p1['coords'][0] - p2['coords'][0])**2
        lon_diff = (p1['coords'][1] - p2['coords'][1])**2
        d = (lat_diff + lon_diff)**0.5 * 111
        distancia_total += d
        
        # LÓGICA DE FRONTEIRA: 
        # Se a Longitude for menor que -57, estamos saindo do Brasil
        # Se a Longitude for menor que -70, entramos no Chile
        lon = p1['coords'][1]
        
        if lon > -57:
            pais = "Brasil"
            preco = PRECO_DIESEL_BR
        elif -70 < lon <= -57:
            pais = "Argentina"
            preco = PRECO_DIESEL_AR
        else:
            pais = "Chile"
            preco = PRECO_DIESEL_CH
            
        custo_trecho = (d / KM_LITRO) * preco
        custo_total_reais += custo_trecho
        
        print(f"📍 Trecho {i+1}: {d:.1f}km em território {pais} (Diesel: R${preco:.2f})")

    print(f"--------------------------------------")
    print(f"🛣️ Distância Total: {distancia_total:.2f} km")
    print(f"💳 Gasto Total Estimado: R$ {custo_total_reais:.2f}")

calcular_custo_internacional(pontos_processados)

In [ ]:
def relatorio_de_carga(pedidos):
    carga_total = sum(p['garrafas'] for p in pedidos)
    carga_atual = carga_total
    
    print(f"📦 MANIFESTO DE CARGA INTERNACIONAL")
    print(f"--------------------------------------")
    print(f"🚀 Saída da Fábrica (Lorena): {carga_total:,} unidades")
    print(f"--------------------------------------")
    
    for p in pedidos:
        if p['tipo'] == 'entrega':
            carga_atual -= p['garrafas']
            print(f"🚚 Entrega em: {p['local']}")
            print(f"   [-] Descarregado: {p['garrafas']:,}")
            print(f"   [=] Carga Restante no Baú: {carga_atual:,} unidades")
            print(f"   ---")

    if carga_atual == 0:
        print("✅ Viagem concluída: Caminhão vazio e pronto para retorno!")
    else:
        print(f"⚠️ Atenção: Sobraram {carga_atual:,} unidades no caminhão.")

relatorio_de_carga(pedidos)

In [ ]:
def logistica_reversa(pontos):
    print(f"🍷 PLANEJAMENTO DE RETORNO: SANTIAGO -> LORENA")
    print(f"----------------------------------------------")
    
    # Cargas de retorno
    retorno = [
        {"item": "Vinhos Premium", "origem": "Santiago, Chile", "valor_venda": 15000},
        {"item": "Azeites Especiais", "origem": "Mendoza, Argentina", "valor_venda": 8000},
        {"item": "Trigo Argentino", "origem": "Buenos Aires, Argentina", "valor_venda": 5000}
    ]
    
    receita_total = sum(i['valor_venda'] for i in retorno)
    
    # Estimativa de custo da volta (já calculamos que a ida deu uns 6k)
    custo_volta = 5800.00 
    lucro_liquido = receita_total - custo_volta
    
    for carga in retorno:
        print(f"📦 Coletando: {carga['item']} em {carga['origem']}")
    
    print(f"----------------------------------------------")
    print(f"💵 Receita Bruta do Frete de Retorno: R$ {receita_total:.2f}")
    print(f"⛽ Custo do Diesel (Volta): R$ {custo_volta:.2f}")
    print(f"💰 LUCRO LÍQUIDO DA OPERAÇÃO RETORNO: R$ {lucro_liquido:.2f}")

logistica_reversa(pontos_processados)

In [ ]:
import matplotlib.pyplot as plt
from fpdf import FPDF
import requests

def dashboard_avancado(pontos):
    # 1. SIMULADOR DE CLIMA (DESAFIO B)
    # Simulando uma consulta em Mendoza (Andes)
    print("❄️ Verificando condições climáticas nos Andes...")
    clima_andes = "Neve" # Poderíamos usar uma API real aqui
    fator_seguranca = 1.25 if clima_andes == "Neve" else 1.0
    
    # 2. CÁLCULO DE LUCRO (DESAFIO C)
    investimento = 6000 * fator_seguranca
    receita_retorno = 28000
    lucro_final = receita_retorno - investimento
    
    # Criando o Gráfico
    categorias = ['Custo Base', 'Custo Clima', 'Receita Retorno']
    valores = [6000, 6000 * (fator_seguranca - 1), receita_retorno]
    
    plt.figure(figsize=(8, 5))
    plt.bar(categorias, valores, color=['red', 'orange', 'green'])
    plt.title('Análise Financeira: Operação Transcontinental')
    plt.ylabel('Valor em R$')
    plt.show()

    # 3. GERADOR DE PDF (DESAFIO A)
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(200, 10, txt="RELATÓRIO DE EXPEDIÇÃO YAKULT - AMÉRICA DO SUL", ln=True, align='C')
    pdf.set_font("Arial", size=12)
    pdf.ln(10)
    pdf.cell(200, 10, txt=f"Distância Percorrida: 3.850 km", ln=True)
    pdf.cell(200, 10, txt=f"Condição nos Andes: {clima_andes}", ln=True)
    pdf.cell(200, 10, txt=f"LUCRO LÍQUIDO FINAL: R$ {lucro_final:.2f}", ln=True)
    
    pdf.output("Relatorio_Logistica_Yakult.pdf")
    print("✅ PDF Gerado com sucesso: 'Relatorio_Logistica_Yakult.pdf'")

dashboard_avancado(pontos_processados)